In [1]:
import pandas as pd
import os

In [3]:
os.chdir(path)

In [4]:
data = pd.read_excel('主订单数据集.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356233 entries, 0 to 356232
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   品牌名     356233 non-null  object
 1   店铺名称    356233 non-null  object
 2   主订单编号   356233 non-null  int64 
 3   用户ID    356231 non-null  object
 4   付款时间    356233 non-null  object
 5   订单状态    356233 non-null  object
 6   实付金额    356233 non-null  int64 
 7   购买数量    356233 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 21.7+ MB


In [5]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 356231 entries, 0 to 356232
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   品牌名     356231 non-null  object
 1   店铺名称    356231 non-null  object
 2   主订单编号   356231 non-null  int64 
 3   用户ID    356231 non-null  object
 4   付款时间    356231 non-null  object
 5   订单状态    356231 non-null  object
 6   实付金额    356231 non-null  int64 
 7   购买数量    356231 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 24.5+ MB


In [6]:
data.describe()

,主订单编号,实付金额,购买数量
count,3.562310e+05,356231.000000,356231.000000
mean,7.346531e+10,166.493085,1.116438
std,1.028359e+05,153.645324,0.817848
min,7.346514e+10,40.000000,1.000000
25%,7.346523e+10,77.000000,1.000000
50%,7.346531e+10,146.000000,1.000000
75%,7.346540e+10,203.000000,1.000000
max,7.346549e+10,29309.000000,150.000000


In [10]:
#use quantile to remove outline value
df = data.copy()
top = df['实付金额'].quantile(0.75) + (df['实付金额'].quantile(0.75)-df['实付金额'].quantile(0.25))*3
bottom = df['实付金额'].quantile(0.25) - (df['实付金额'].quantile(0.75)-df['实付金额'].quantile(0.25))*3
print(top, bottom)

581.0 -301.0


In [11]:
df = df.loc[df['实付金额'] <= top,:]
df.describe()

,主订单编号,实付金额,购买数量
count,3.533680e+05,353368.000000,353368.000000
mean,7.346531e+10,160.609150,1.103861
std,1.027865e+05,99.092419,0.645977
min,7.346514e+10,40.000000,1.000000
25%,7.346523e+10,77.000000,1.000000
50%,7.346531e+10,146.000000,1.000000
75%,7.346540e+10,199.000000,1.000000
max,7.346549e+10,581.000000,145.000000


In [14]:
# process buy number
BuyNum = df['购买数量'].value_counts().reset_index()
BuyNum.columns = ['BuyNum', 'Order']
BuyNum['OrderPercent'] = BuyNum['Order']/BuyNum['Order'].sum()
BuyNum['OrderPercentCumsum'] = BuyNum['OrderPercent'].cumsum()
BuyNum

,BuyNum,Order,OrderPercent,OrderPercentCumsum
0,1,329739,0.933132,0.933132
1,2,14216,0.040230,0.973362
2,3,7998,0.022634,0.995996
3,4,687,0.001944,0.997940
4,5,449,0.001271,0.999210
5,6,152,0.000430,0.999641
6,7,35,0.000099,0.999740
7,10,26,0.000074,0.999813
8,8,12,0.000034,0.999847
9,9,11,0.000031,0.999878


In [15]:
#so we can keep BuyNum <=7, which take 99.974%
df = df.loc[df['购买数量'] <= 7, :]

In [16]:
#process 订单状态
df['订单状态'].value_counts()

订单状态
交易成功                 289519
付款以前，卖家或买家主动关闭交易      37269
付款以后用户退款成功，交易自动关闭     26473
等待买家确认收货,即:卖家已发货         15
Name: count, dtype: int64

In [18]:
#only keep sucess
df = df.loc[df['订单状态'] == '交易成功', :]
print(len(df))

289519


In [25]:
#apply 28 rule to the data
group_28 = df.groupby('用户ID')['实付金额'].sum().sort_values(ascending=False).reset_index()

,用户ID,实付金额,order_index
0,uid135476531,40044,1
1,uid135460679,35543,2
2,uid135467627,30916,3
3,uid135473980,24158,4
4,uid135465649,21015,5


In [30]:
#get the value of top 4% and 20%
print(group_28['实付金额'].quantile(0.96), group_28['实付金额'].quantile(0.8))

1038.0 460.0


In [31]:
#label data
def AddLabel28(x):
    if x >= 1028:
        return 'AdcancedUser'
    elif x >= 460:
        return 'MiddleUser'
    else:
        return 'LowUser'
group_28['UserLevel'] = group_28['实付金额'].apply(AddLabel28)
group_28.head()

,用户ID,实付金额,order_index,UserLevel
0,uid135476531,40044,1,AdcancedUser
1,uid135460679,35543,2,AdcancedUser
2,uid135467627,30916,3,AdcancedUser
3,uid135473980,24158,4,AdcancedUser
4,uid135465649,21015,5,AdcancedUser


In [33]:
#check data by level
group_28_level = group_28.groupby('UserLevel')['实付金额'].sum().reset_index()
group_28_level['Rate'] = group_28_level['实付金额']/group_28_level['实付金额'].sum()
group_28_level.head()

,UserLevel,实付金额,Rate
0,AdcancedUser,9196490,0.202028
1,LowUser,22313038,0.490172
2,MiddleUser,14011305,0.307800
